In [1]:
%load_ext autoreload
%autoreload 2

from IPython.display import display_html, HTML, display
css = !cat styles.css
display(HTML(''.join(css)))

import numpy as np

import display_helpers

In [2]:
from erafixer import EraFixer, COL_LOOKUP

In [28]:
# Excel file
# fn = 'ERA2018_Pubs_02_27-Oct-17.xlsx'
fn = 'all.xlsx'

### Create EraFixer instance

In [43]:
erafixer = EraFixer(fn=fn, sheet_index=1, verbose=True)

Parsing file all.xlsx
Using sheet index 1 - Tab 1 for 02


In [44]:
erafixer.set_author_discipline('Zvyagin', 'biophotonics')
erafixer.set_author_discipline('Schwab', 'astro')
erafixer.set_author_discipline('Gee', 'astro')

Setting discipline to 'biophotonics' for 'Zvyagin'
Found 0 matches for 'Zvyagin' with empty discipline
Setting discipline to 'astro' for 'Schwab'
Found 32 matches for 'Schwab' with empty discipline
Setting discipline to 'astro' for 'Gee'
Found 1 matches for 'Gee' with empty discipline


In [45]:
erafixer.set_journal_discipline('astrophysical', 'astro')
erafixer.set_journal_discipline('geophysical', 'geo')
erafixer.set_journal_discipline('geophysics', 'geo')
erafixer.set_journal_discipline('optics express', 'photonics')

Setting discipline to 'astro' for 'astrophysical'
Found 0 matches for 'astrophysical' with empty discipline
Setting discipline to 'geo' for 'geophysical'
Found 0 matches for 'geophysical' with empty discipline
Setting discipline to 'geo' for 'geophysics'
Found 0 matches for 'geophysics' with empty discipline
Setting discipline to 'photonics' for 'optics express'
Found 0 matches for 'optics express' with empty discipline


In [7]:
erafixer.save()

Writing sheet 'Legend' to all.xlsx
Writing sheet 'Tab 1 for 02' to all.xlsx
Writing sheet 'Tab 2 for 02' to all.xlsx


'all.xlsx'

In [8]:
erafixer.split_disciplines('disc')

Writing dataframe to disc_geo.xlsx with 4 records
Writing dataframe to disc_biophotonics.xlsx with 39 records
Writing dataframe to disc_astro.xlsx with 192 records
Writing dataframe to disc_photonics.xlsx with 94 records


['disc_geo.xlsx',
 'disc_biophotonics.xlsx',
 'disc_astro.xlsx',
 'disc_photonics.xlsx']

#### Carry Forward FORCS

In [9]:
astro_erafixer = EraFixer(fn='disc_astro.xlsx', verbose=True)

Parsing file disc_astro.xlsx
Using sheet index 0 - Sheet1


In [10]:
astro_erafixer.carry_forward_forcs()

Found 192 matches for HANDLED=0
Copying 2015 FOR codes to 2018 for unhandled rows


In [11]:
astro_erafixer.save()

Writing sheet 'Sheet1' to disc_astro.xlsx


'disc_astro.xlsx'

#### Apply FORC_STRING

In [24]:
astro_erafixer = EraFixer(fn='disc_astro.xlsx', verbose=True)

Parsing file disc_astro.xlsx
Using sheet index 0 - Sheet1


In [25]:
# forc_string = '0201'
# forc_string = '0201:40,0203'
# forc_string = '0201:40,0203:60'
forc_string = '0201:40,0203:30,0204'
# forc_string = '0201:40,0203:25,0204:30'

# justify_string = None
justify_string = 'Development of Raman lasers'

In [23]:
astro_erafixer.set_forc_string(forc_string, justify_string=justify_string, )

Applying FORC_STRING '0201:40,0203:30,0204'
Found 163 matches for '0' with HANDLED=0
Not all codes present and not justify, marking HANDLED=ClawbackNeeded
Not all codes present and not justify, marking HANDLED=ClawbackNeeded
Not all codes present and not justify, marking HANDLED=ClawbackNeeded
Not all codes present and not justify, marking HANDLED=ClawbackNeeded
Not all codes present and not justify, marking HANDLED=ClawbackNeeded
Not all codes present and not justify, marking HANDLED=ClawbackNeeded
Not all codes present and not justify, marking HANDLED=ClawbackNeeded
Not all codes present and not justify, marking HANDLED=ClawbackNeeded
Not all codes present and not justify, marking HANDLED=ClawbackNeeded
Not all codes present and not justify, marking HANDLED=ClawbackNeeded
Not all codes present and not justify, marking HANDLED=ClawbackNeeded
Not all codes present and not justify, marking HANDLED=ClawbackNeeded
Not all codes present and not justify, marking HANDLED=ClawbackNeeded
Not a

In [47]:
matches.keys()

dict_keys([23, 29, 30, 31, 32, 33, 34, 35, 36, 37, 38, 39, 41, 42, 43, 44, 45, 46, 47, 48, 49, 50, 51, 52, 54, 55, 56, 57, 58, 59, 64, 66, 67, 68, 69, 70, 72, 73, 74, 75, 76, 77, 79, 80, 81, 82, 83, 84, 86, 87, 88, 89, 90, 91, 92, 93, 94, 95, 96, 98, 99, 100, 101, 102, 103, 104, 105, 106, 107, 108, 109, 110, 111, 112, 113, 114, 115, 116, 117, 118, 119, 120, 146, 152, 153, 154, 155, 156, 157, 158, 159, 160, 161, 162, 163, 164, 165, 166, 167, 168, 169, 170, 171, 172, 173, 175, 176, 177, 178, 179, 181, 182, 183, 184, 185, 186, 187, 188, 189, 190])

In [48]:
code1, code1_perc, code2, code2_perc, code3, code3_perc = astro_erafixer._parse_forc_string(forc_string)
    
print(code1, code2, code3)
print(code1_perc, code2_perc, code3_perc)

0201 0203 0204
40.0 30.0 30.0


#### Test author match

We want to make sure we are matching the correct authors. The author should be supplied as the full last name of the author in question. Names are not stored correctly in the excel sheet (should be in UTF-8) but will still match on bad characters. See top name below.

In [46]:
authors = [
    'JelÃ­nkovÃ¡',
    'gee',
    'McGee',
    '    sTeEl ',
    'steel',
    'steele',
    'casteels',
    'xia',
    'xiao',
    'Zvyagin'
]

for author in authors:
    
    row_match = erafixer.get_matching_rows(author, 'AUTHORS', blank_discipline=False)
    
    if(row_match):
        display_helpers.show_matches(erafixer, row_match, author)

Found 4 matches for AUTHORS=JelÃ­nkovÃ¡


Full Name Jelã­Nkovã¡ H.     Authors: JelÃ­nek M.;Kitzler O.;JelÃ­nkovÃ¡ H.;Â¿ulc J.;NÂ¿mec M. Title: CVD-diamond external cavity nanosecond Raman laser operating at 1.63 Â¿m pumped by 1.34 Â¿m Nd:YAP laser ---   Authors: JelÃ­nkovÃ¡ Helena;Kitzler OndÂ¿ej;JelÃ­nek Michal;Â¿ulc Jan;NÂ¿mec Michal;KubeÂ¿ek VÃ¡clav Title: Diamond Raman laser in eye safe region ---   Authors: Kitzler OndÂ¿ej;JelÃ­nkovÃ¡ Helena;Â¿ulc Jan;KoubÃ­kovÃ¡ Lucia;NÂ¿mec Michal;Nejezchleb Karel;Â¿koda VÃ¡clav Title: High energy intracavity pumped eye-safe BaWOÂ¿ Raman laser ---   Authors: JelÃ­nek Michal;Kitzler OndÂ¿ej;JelÃ­nkovÃ¡ Helena;Â¿ulc Jan;NÂ¿mec Michal;KubeÂ¿ek VÃ¡clav Title: CVD-diamond external cavity Raman laser operating at 1632 nm ---

Found 1 matches for AUTHORS=gee


Full Name Gee Wilfred T.     Authors: Gee Wilfred T.;Guyon Olivier;Walawender Josh;Jovanovic Nemanja;Boucher Luc Title: Project PANOPTES a citizen-scientist exoplanet transit survey using commercial digital cameras ---

Found 2 matches for AUTHORS=McGee


...

Full Name Xia Ke Yu     Authors: Xia Ke Yu ; Brennen Gavin Keith ; Ellinas Demosthenes ; Twamley Jason Title: Deterministic generation of an on-demand Fock state ---   Authors: Xia Ke Yu ; Twamley Jason Title: All-optical switching and router via the direct quantum control of coupling between cavity modes ---   Authors: Xia Ke Yu ; Zhao Nan ; Twamley Jason Title: Detection of a weak magnetic field via cavity-enhanced Faraday rotation ---   Authors: Xia Keyu;Twamley Jason Title: Generating spin squeezing states and Greenberger-Horne-Zeilinger entanglement using a hybrid phonon-spin ensemble in diamond ---

Found 3 matches for AUTHORS=xiao


Full Name Xiao Ting     Authors: Bundy Kevin ; Bershady Matthew A. ; Law David ; Yan Renbin ; Drory Niv ; MacDonald Nicholas R. ; Wake David A. ; Cherinka Brian A. ; Sanchez-Gallego Jose Ramon ; Weijmans Anne-Marie ; Thomas Daniel B. ; Tremonti Christy A ; Masters Karen Louise ; Coccato Lodovico ; Diamond-Stanic Aleksandar M. ; Aragon-Salamanca Alfonso ; Avila-Reese Vladimir ; Badenes Carlos ; Falcon-Barroso Jesus ; Belfiore Francesco ; Bizyaev Dmitriy V ; Blanc Guillermo A. ; Bland-Hawthorn Jonathan ; Blanton Michael R ; Brownstein Joel R. ; Byler Nell ; Cappellari Michele ; Conroy Charlie ; Dutton Aaron A. ; Emsellem Eric ; Etherington James ; Frinchaboy Peter M. ; Fu Hai ; Gunn James E ; Harding Paul ; Johnston Evelyn J. ; Kauffmann Guinevere ; Kinemuchi Karen ; Klaene Mark A. ; Klaene Mark A. ; Leauthaud Alexie ; Li Cheng ; Lin Lihwai ; Maiolino Roberto ; Malanushenko Viktor ; Malanushenko Elena ; Mao Shude ; Maraston C ; McDermid Richard ; Merrifield Michael R. ; Nichol Robert C. ; Oravetz Daniel J. ; Pan Kaike ; Parejko John K. ; Sanchez Sebastian F. ; Schlegel David J ; Simmons Audrey E. ; Steele Oliver ; Steinmetz Matthias ; Thanjavur Karun ; Thompson Benjamin A. ; Tinker Jeremy L. ; van den Bosch Remco C. E. ; Westfall Kyle B. ; Wilkinson David ; Wright Shelley ; Xiao Ting ; Zhang Kai Title: Overview of the SDSS-IV MaNGA survey : mapping nearby galaxies at Apache Point Observatory ---   Authors: Che Xiao;Sturmann Laszlo;Monnier John D.;ten Brummelaar Theo A.;Sturmann Judit;Ridgway Stephen T.;Ireland Michael J.;Turner Nils H.;McAlister Harold A. Title: The CHARA array adaptive optics I common-path optical and mechanical design, and preliminary on-sky results ---   Authors: Ten Brummelaar Theo A.;Sturmann Laszlo;Sturmann Judit;Ridgway Stephen T.;Monnier John D.;Ireland Michael J.;Che Xiao;McAlister Harold A.;Turner Nils H.;Tuthill P. G. Title: Adaptive optics for the CHARA array ---

Found 39 matches for AUTHORS=Zvyagin


Full Name Zvyagin Andrei     Authors: Zvyagin Andrei ; Song Zhen ; Nadort Annemarie ; Sreenivasan Varun Kumaraswamy Annayya Chetty ; Deyev Sergey M Title: Luminescent nanomaterials for molecular-specific cellular imaging ---   Authors: Song Zhen ; Kelf Timothy ; Sanchez Washington ; Roberts Micheal S ; Ricka Jaro ; Frenz Martin ; Zvyagin Andrei Title: Characterization of optical properties of ZnO nanoparticles for quantitative imaging of transdermal transport ---   Authors: Zvyagin Andrei ; Manson N B Title: Optical and spin properties of nitrogen-vacancy color centers in diamond crystals, nanodiamonds, and proximity to surfaces ---   Authors: Sreenivasan Varun Kumaraswamy Annayya Chetty ; Kim Eun Ju ; Goodchild Ann Kathleen ; Connor Mark Andrew ; Zvyagin Andrei Title: Targeting somatostatin receptors using in situ-bioconjugated fluorescent nanoparticles ---